In [9]:
from document_parser import get_chunks_from_github
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
from IPython.display import display, HTML


In [2]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')
# all-MiniLM-L6-v2
# all-mpnet-base-v2


In [3]:
docs = [
    "https://github.com/virtual-labs/app-exp-create-web/blob/master/docs/developer-doc.md",
    "https://github.com/virtual-labs/app-exp-create-web/blob/master/docs/user-doc.md",
    "https://github.com/virtual-labs/app-vlabs-pwa/blob/main/docs/tech_guide.md",
    "https://github.com/virtual-labs/app-vlabs-pwa/blob/main/docs/user_manual.md"
     ]
data = get_chunks_from_github(*docs)

[{'content': 'Virtual Labs Content Development Platform - Developer Document :: Virtual Labs Content Development Platform - Developer Document :: This document serves as a guide for developers working on the Virtual Labs Content Development Platform project. It provides an overview of the project, development guidelines, and deployment instructions.', 'payload': {'page_title': 'Virtual Labs Content Development Platform - Developer Document', 'heading': 'Virtual Labs Content Development Platform - Developer Document', 'text': 'Virtual Labs Content Development Platform - Developer Document :: Virtual Labs Content Development Platform - Developer Document :: This document serves as a guide for developers working on the Virtual Labs Content Development Platform project. It provides an overview of the project, development guidelines, and deployment instructions.', 'url': 'https://github.com/virtual-labs/app-exp-create-web/blob/master/docs/developer-doc.md#virtual-labs-content-development-pl

In [4]:
qdrant = QdrantClient(":memory:")

In [5]:
qdrant.recreate_collection(
    collection_name="my_doc",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.DOT
    )
)

True

In [6]:
qdrant.upload_records(
    collection_name="my_doc",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["content"]).tolist(),
            payload=doc["payload"]
        ) for idx, doc in enumerate(data)
    ]
)

In [7]:
queries = [
	"what are future update and releases for content development platform",
	"how to modify experiment after creating it",
	"getting authentication error in content development platform",
	"how do i search experiments in pwa",
	"how do i apply filters in vlabs pwa",
	"how do i apply filters on search in vlabs pwa",
	"what is google's revenue in 2022",
	"what is my name",
	"Virtual Labs Content Development Platform - Developer Document"
]
# argmode, lierate programming
hits = qdrant.search(
	collection_name="my_doc",
	query_vector=encoder.encode(
		queries[1]).tolist(),
    limit=10
    # all-mpnet-base-v2
)
print(len(hits))
table_contents = []
for hit in hits:
	table_contents.append({
		"url": hit.payload["url"],
		"score": hit.score,
		"heading": hit.payload["heading"],
		"document": hit.payload["page_title"],
		"text": hit.payload["text"].split("::")[2].strip(),
	})
df = pd.DataFrame(table_contents)
df["url"] = df["url"].apply(lambda link: f'<a href="{link}">Visit Link</a>')
df["text"] = df["text"].apply(lambda link: link.replace("\n","<br/>"))
display(HTML(df.to_html(escape=False)))


10


,url,score,heading,document,text
0,Visit Link,0.575425,Creating an Experiment,User Document - Virtual Labs Content Development Platform,". Visit the Virtual Labs Content Development Platform landing page.. In the navigation bar, click on the ""Create"" tab.. Read the instructions provided on the page. - Make sure you have a GitHub. account. If you don't have one, you can sign up for free at GitHub - Create experiment: - Creating Experiment in Virtual Labs: Follow the instructions in the engineers forum repository to create an experiment in Virtual Labs: Engineers forum. - Creating Experiment in your organization: - Create the experiment repository with experiment template. - Name format for repository should follow the given format `exp--` e.g. exp-geometry-optimization-molecules-iiith - Must select `Include all branches` while creating repository from this template. - After creating repository setup Secret Token. Once you have created the experiment repository, proceed to the next section for modifying the experiment."
1,Visit Link,0.532702,Modifying Experiment Components,User Document - Virtual Labs Content Development Platform,"The Virtual Labs Content Development Platform provides two editors to create experiment components: Markdown editor and rich text editor. Follow these steps to create experiment components:. Navigate to the specific component you want to edit, such as Aim, Theory, etc.. Choose the Markdown editor or the rich text editor to write and structure the content.. Save your changes to update the experiment component."
2,Visit Link,0.499306,Modifying an Experiment,User Document - Virtual Labs Content Development Platform,". On the Virtual Labs Content Development Platform landing page, click on the ""Modify"" tab in the navigation bar.. In the form, enter the organisation name and the experiment repository name.. Click the ""Modify"" button.. Authenticate yourself using your GitHub account credentials.. After authentication, you will be redirected to content management page."
3,Visit Link,0.486494,Experiment Cards,User Manual : Virtual Labs App,"* Each experiment is represented by a card containing the following information: * Discipline: The field of study to which the experiment belongs. It helps you categorise and identify experiments based on their areas of focus. * Description: A brief overview of the experiment. The description provides a concise summary of the experiment's purpose and objectives, giving you an idea of what to expect. * Link: A clickable link that takes you to the main experiment page. By clicking on the link, you can access the detailed information, instructions, and materials required for the experiment."
4,Visit Link,0.382317,Search Functionality,User Manual : Virtual Labs App,"* The app offers a powerful search feature that allows you to find experiments based on specific criteria. To perform a search, follow these steps: * Experiment Name: Enter the name of the experiment you are looking for in the search bar. The search results will display experiments matching the entered name. * Tags: You can search using relevant keywords or tags associated with experiments. Enter the desired tag in the search bar, and the results will show experiments related to that tag. * Lab Name: Find experiments created by a specific institute or laboratory by typing the name of the institute in the search bar. The search results will include experiments associated with that institute."
5,Visit Link,0.378971,Introduction,User Manual : Virtual Labs App,"* The Virtual-Labs app is designed to provide users with access to a wide range of experiments from various disciplines. These experiments are presented in the form of cards, offering basic information and a clickable link to access the main experiment page. The app also includes a search function and filters to help you find specific experiments based on your preferences.It is hosted on app.vlabs.ac.in ."
6,Visit Link,0.367513,Conclusion,User Manual : Virt